In [1]:
!pip install sentence-transformers faiss-gpu langchain langchain_huggingface bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.9 MB/s eta 0:00:00


In [2]:
!pip uninstall -y torchvision
!pip install torchvision
!pip install transformers --upgrade

Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [3]:
import os
from sentence_transformers import SentenceTransformer
import json
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
# # Inisialisasi model embedding
# embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Disable ONEDNN optimizations
# os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

In [7]:
# Memuat data resep dari file JSON
with open("recipes.json", "r") as file:
    recipes = json.load(file)

# Bagian Baru

In [8]:
index = faiss.read_index("faiss_index.bin")

In [9]:
def load_allergen_db(file_path):
    with open(file_path, 'r') as file:
        allergen_db = json.load(file)
    return allergen_db

allergen_db_file = "allergen_db.json"

allergen_db = load_allergen_db(allergen_db_file)

In [ ]:
# # Membuat embedding dari data resep
# embeddings = embedding_model.encode(recipes)
# np.save("document_embeddings.npy", embeddings)

In [ ]:
# # Membuat indeks FAISS
# embeddings = np.load("document_embeddings.npy").astype('float32')
# index = faiss.IndexFlatL2(embeddings.shape[1])
# index.add(embeddings)

# # Menyimpan indeks FAISS
# faiss.write_index(index, "faiss_index.bin")

In [ ]:
# # Memuat data resep dari file JSON
# with open("recipes.json", "r") as file:
#     recipes = json.load(file)

# # Membuat embedding dari data resep
# embeddings = embedding_model.encode(recipes)
# index = faiss.IndexFlatL2(embeddings.shape[1])
# index.add(embeddings)

# faiss.write_index(index, "faiss_index.bin")

In [ ]:
# # Memuat indeks FAISS
# index = faiss.read_index("faiss_index.bin")

# # Membuat query embedding
# query = embedding_model.encode(["Gado-Gado Siram"]).astype('float32')

# # Mencari 5 resep terdekat
# D, I = index.search(query, k=5)
# print("Distances:", D)
# print("Indices:", I)

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [12]:
class FAISSRetriever:
    def __init__(self, index, recipes, model):
        self.index = index
        self.recipes = recipes
        self.model = model

    def __call__(self, question: str):
        # Ensure that the question is a string
        if not isinstance(question, str):
            raise ValueError("The input must be a string")

        # Directly receive a question string
        query_embedding = self.model.encode([question]).astype('float32')
        D, I = self.index.search(query_embedding, k=3)  # Retrieve top 3 results

        # Retrieve the documents based on the indices from FAISS
        retrieved_docs = [self.recipes[i] for i in I[0]]

        # Process each document (which is a dict) and extract relevant information
        processed_docs = []
        for doc in retrieved_docs:
            # You can customize this depending on what parts of the doc you want to include
            recipe_name = doc.get('nama', 'No name provided')
            bahan = doc.get('bahan', {})
            cara_membuat = doc.get('cara_membuat', [])
            porsi = doc.get('porsi', 'Unknown')

            # Format the content into a readable string
            formatted_doc = f"Recipe: {recipe_name}\n"
            formatted_doc += f"Serving Size: {porsi} servings\n"

            # Check if bahan is a dictionary
            if isinstance(bahan, dict):
                # Adding the ingredients
                for category, ingredients in bahan.items():
                    formatted_doc += f"{category.capitalize()}:\n"
                    for ingredient in ingredients:
                        formatted_doc += f"- {ingredient}\n"
            else:
                # If bahan is not a dictionary, provide a default message
                formatted_doc += "Saat ini resep belum tersedia.\n"

            # Adding the cooking instructions
            if cara_membuat:
                formatted_doc += "\nCooking Instructions:\n"
                for step in cara_membuat:
                    formatted_doc += f"- {step}\n"

            processed_docs.append(formatted_doc)

        # Join all the processed documents into one string to return as the final result
        return "\n\n".join(processed_docs)

# Instantiate the retriever
retriever = FAISSRetriever(index, recipes, model)

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="unsloth/Llama-3.2-1B-Instruct",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        return_full_text=False,
    ),
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
# Pilih model 2
model_name = "aisingapore/gemma2-9b-cpt-sea-lionv3-instruct"  # Model yang Anda gunakan

# Konfigurasi untuk 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use FP16 for computation
    bnb_4bit_use_double_quant=True,  # Double quantization for better compression
    bnb_4bit_quant_type="nf4"  # Normalized float (NF4) quantization
)

# Inisialisasi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=False)

# Load model dengan konfigurasi quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,  # Terapkan quantization
    device_map="auto",  # Model akan dimuat ke GPU jika tersedia
    local_files_only=False
)

# Tentukan perangkat
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/46.7k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernor

In [15]:
# Prompt untuk few-shot prompting

def generate_nutrition_prompt(response):
    base_prompt = """
Tugas Anda adalah membaca resep masakan, mengidentifikasi bahan, dan memberikan estimasi kandungan gizi (kalori, protein, karbohidrat, lemak) berdasarkan porsi yang disediakan.
Berikut adalah beberapa contohnya:

Resep:
Nama: Pancake Sederhana
Bahan:
- 100 gr tepung terigu
- 2 butir telur
- 100 ml susu
- 1 sdm gula
- 1 sdm mentega
- 1 sdt baking powder
Porsi: 4

Estimasi Kandungan Gizi (per porsi):
- Kalori: 150 kcal
- Protein: 6 g
- Karbohidrat: 18 g
- Lemak: 5 g

Resep:
Nama: Nasi Goreng Telur
Bahan:
- 200 gr nasi
- 1 butir telur
- 2 sdm minyak goreng
- 1 sdm kecap manis
Porsi: 2

Estimasi Kandungan Gizi (per porsi):
- Kalori: 250 kcal
- Protein: 5 g
- Karbohidrat: 30 g
- Lemak: 10 g

Resep:
Nama: Sup Sayur Sehat
Bahan:
- 100 gr wortel
- 100 gr kentang
- 50 gr brokoli
- 800 ml air
- 1 sdt garam
Porsi: 4

Estimasi Kandungan Gizi (per porsi):
- Kalori: 70 kcal
- Protein: 2 g
- Karbohidrat: 12 g
- Lemak: 0.5 g

Resep:
Nama: Ayam Panggang Madu
Bahan:
- 150 gr daging ayam
- 1 sdm madu
- 1 sdm kecap asin
- 1 sdt lada hitam
Porsi: 3

Estimasi Kandungan Gizi (per porsi):
- Kalori: 160 kcal
- Protein: 15 g
- Karbohidrat: 5 g
- Lemak: 8 g

Resep:
Nama: Mie Goreng Sayuran
Bahan:
- 200 gr mie
- 50 gr kol
- 50 gr wortel
- 2 sdm minyak goreng
- 1 sdm saus tiram
Porsi: 2

Estimasi Kandungan Gizi (per porsi):
- Kalori: 300 kcal
- Protein: 7 g
- Karbohidrat: 40 g
- Lemak: 10 g

Resep:
Nama: Salad Buah
Bahan:
- 100 gr apel
- 100 gr anggur
- 100 gr pepaya
- 50 ml yogurt
Porsi: 2

Estimasi Kandungan Gizi (per porsi):
- Kalori: 120 kcal
- Protein: 2 g
- Karbohidrat: 25 g
- Lemak: 2 g

Resep:
Nama: Es Campur
Bahan:
- 100 gr kelapa muda
- 50 gr cincau
- 100 ml sirup
- 200 ml air
- 50 gr es serut
Porsi: 4

Estimasi Kandungan Gizi (per porsi):
- Kalori: 90 kcal
- Protein: 1 g
- Karbohidrat: 20 g
- Lemak: 0 g

Resep berikutnya:
"""
    complete_prompt = base_prompt + response + """
Estimasi Kandungan Gizi (per porsi):
"""
    return complete_prompt


In [16]:
# question = "Saya ingin membuat empal jawa barat. Berikan saya tahapan memasaknya dan bahan-bahan lengkapnya!"
question = "Saya ingin membuat gado gado padang. Berikan saya tahapan memasaknya dan bahan-bahan lengkapnya!"

In [17]:
# Example query model 1
response = rag_chain.invoke(question)
# print(response)

In [18]:
# model 2
# Tokenisasi input
inputs = tokenizer.encode(generate_nutrition_prompt(response), return_tensors="pt").to(device)

model.config.max_batch_size = 1  # Atur batch size menjadi lebih kecil

# Generasi teks dengan model
outputs = model.generate(
    inputs,
    max_new_tokens=80,  # Reduce to minimal required output
    temperature=0.7,    # Balance creativity and determinism
    top_k=50,           # Sample from top-k predictions
    top_p=0.9,          # Use nucleus sampling for better efficiency
    num_beams=1,        # Avoid beam search for lower memory usage
    early_stopping=True
)

# Decode hasil output
# result = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(result.split("Resep berikutnya:")[-1].strip().replace("*", ""))

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


In [19]:
# model 3
import re

def detect_allergens(input_text, allergen_db):
    detected_allergens = {}
    input_text = input_text.lower()

    words = re.findall(r'\b\w+\b', input_text)

    for key, allergen in allergen_db.items():
        key_lower = key.lower()
        if key_lower in words:
            detected_allergens[key] = allergen

    return detected_allergens


allergen_result = detect_allergens(response, allergen_db)

# if allergen_result:
#     print("Bahan-bahan yang terdapat pada resep mengandung allergen berikut:")
#     for ingredient, allergen in allergen_result.items():
#         print(f"- {allergen} ({ingredient})")
# else:
#     print("Tidak ada allergen terdeteksi.")


In [20]:
def process_recipe_with_nutrition_and_allergens(question, tokenizer, model, allergen_db):

    response = rag_chain.invoke(question)

    # Tokenisasi input
    inputs = tokenizer.encode(generate_nutrition_prompt(response), return_tensors="pt").to(device)

    # Konfigurasi model
    model.config.max_batch_size = 1  # Atur batch size menjadi lebih kecil

    # Generasi teks dengan model
    outputs = model.generate(
        inputs,
        max_new_tokens=80,  # Reduce to minimal required output
        temperature=0.7,    # Balance creativity and determinism
        top_k=50,           # Sample from top-k predictions
        top_p=0.9,          # Use nucleus sampling for better efficiency
        num_beams=1,        # Avoid beam search for lower memory usage
        early_stopping=True
    )

    # Decode hasil output
    nutrition_info = tokenizer.decode(outputs[0], skip_special_tokens=True)
    nutrition_info = nutrition_info.split("Resep berikutnya:")[-1].strip().replace("*", "")

    allergen_result = detect_allergens(response, allergen_db)

    result = f"{nutrition_info}\n\n"

    if allergen_result:
        result += "Bahan-bahan berikut mengandung allergen:\n"
        for ingredient, allergen in allergen_result.items():
            result += f"- {ingredient.capitalize()}: {allergen}\n"
    else:
        result += "Tidak ada allergen yang terdeteksi dalam bahan-bahan.\n"

    return result

In [21]:
question = "Saya ingin membuat sebuah masakan Gudeg Ceker khas Jogjakarta. Berikan saya tahapan memasaknya dan bahan-bahan lengkapnya!"
print(process_recipe_with_nutrition_and_allergens(question, tokenizer, model, allergen_db))

Here are the steps and ingredients for making Gudeg Ceker (Jogjakarta):

Tahapan Memasak:

1. Cuci nangka muda dan rebus sebentar dengan air kelapa untuk menghilangkan kulit dan biji.
2. Siapkan panci, alasi dasar panci dengan daun jati, dan tuang nangka muda dan bumbu halus ke dalam panci.
3. Tambahkan sisa nangka, telur, sisa bumbu, gula merah, daging sapi, dan ceker ayam ke dalam panci.
4. Tuang santan, tutup panci, masak dengan api sedang hingga gudeg matang dan ceker lunak.
5. Angkat dan sajikan dengan nasi hangat.

Bahan-Bahan Lengkap:

 750 gr nangka muda
 500 ml air kelapa
 750 ml santan kental dari 2 butir kelapa
 4 butir telur, rebus matang, kupas
 2 lembar daun jati
 150 gr gula merah
 6 lembar daun salam
 4 cm lengkuas, memarkan
 100 gr daging sapi, potong-potong
 10 ceker ayam, cuci bersih
 Bumbu halus:
	+ 12 butir bawang merah
	+ 6 siung bawang putih
	+ 8 butir kemiri, sangrai
	+ 1 sdm ketumbar, sangrai
	+ 1 sdt garam

Catatan: Pastikan untuk menggunakan nangka muda segar